In [1]:
import timeit
import matplotlib.pyplot as plt
import pickle
from mnist import MNIST
from time import time

In [2]:
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255)
print("Done.")

Loading dataset....
Done.


In [3]:
def one_hot_encode(labels):
    y_inp=np.zeros((len(labels),10))
    for ind,val in enumerate(labels):
        y_inp[ind][val]=1
    return y_inp

In [4]:
def batch_gen(st=0,size=20,validate=False):
    st=st%60000
    if validate:
        X=X_test[st:st+size].reshape(size,784)
        labels=y_test[st:st+size]
    else:
        X=X_train[st:st+size].reshape(size,784)
        labels=y_train[st:st+size]
    y=one_hot_encode(labels)
    return X,y

In [5]:
from nnet.network import Sequential,layers
from nnet.layers import conv2d,max_pool,flatten,dense,dropout,Activation
from nnet import functions
from nnet import optimizers
import numpy as np

Seed: 917
Seed: 599


In [6]:
model=Sequential()
model.add(dense(128,input_shape=(784),activation=functions.sigmoid))
model.add(dense(32,activation=functions.sigmoid))
model.add(dense(10,activation=functions.sigmoid))
# model.add(Activation(functions.sigmoid))

In [7]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
0 input_layer(InputLayer) (None, 784)                echo             0
__________________________________________________________________________________________
1 dense(dense)            (None, 128)                sigmoid          100480
__________________________________________________________________________________________
2 dense(dense)            (None, 32)                 sigmoid          4128
__________________________________________________________________________________________
3 dense(dense)            (None, 10)                 sigmoid          330
Total Params: 104,938
Trainable Params: 104,938
Non-trainable Params: 0


In [8]:
model.compile(optimizer=optimizers.iterative,loss=functions.cross_entropy_with_logits,learning_rate=0.1)

In [9]:
btsz=2
inp,y_inp=batch_gen(0,size=btsz)
logits=model.fit(inp,y_inp)

In [10]:
X_inp=inp
labels=y_inp
for obj in model.sequence:
	X_inp=obj.forward(X_inp)
err=model.del_loss(X_inp,labels)

In [11]:
model.predict(inp)-y_inp

array([[ 0.49914868,  0.2828483 ,  0.28902254,  0.29132955,  0.28561171,
        -0.49733967,  0.28262135,  0.27966944,  0.28626384,  0.2899833 ],
       [-0.500862  ,  0.28287664,  0.2890462 ,  0.29134765,  0.28563581,
         0.50267163,  0.28262781,  0.27969964,  0.28628309,  0.29001359]])

In [12]:
i=model.lenseq_m1
for obj in model.sequence[::-1]:
	err=obj.backprop(err,layer=i)
	i-=1
model.optimizer(model.sequence,model.learning_rate,model.beta)

In [13]:
logits.argmax(axis=1),y_inp.argmax(axis=1)

(array([3, 3]), array([5, 0]))

In [14]:
logits.repeat(10,axis=0).shape

(20, 10)

In [15]:
def validate():
    vdsz=2000
    dvme=10000//vdsz
    acc=0
    cross_entropy_loss=0
    for smpl in range(0,10000,vdsz):
        print("\rCalculating Validation acc...",smpl//vdsz+1,end='')
        inp,y_inp=batch_gen(smpl,size=vdsz,validate=True)
        logits=model.predict(inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc+=100*(ans==cor).mean()
        cross_entropy_loss+=model.loss(logits,labels=y_inp).mean()*10
    print("\rValidation Acc: {:.3f} %        Val loss: {:.8f}".format(acc/dvme,cross_entropy_loss/dvme))
    logits=model.predict(inp[:1])

In [16]:
def run():
    st=0
    btsz=16
    acc_tn=0
    _cc=0
    loss_tn=0
    while st<=60000:
        sam_tm=time()
        perc=st/600
        ck=np.random.randint(0,60000-btsz)
        inp,y_inp=batch_gen(ck,size=btsz)
        logits=model.fit(inp,y_inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc=100*(ans==cor).mean()
        cross_entropy_loss=model.loss(logits=logits,labels=y_inp).mean()*10
        acc_tn+=acc
        _cc+=1
        loss_tn+=cross_entropy_loss
        acc=acc_tn/_cc
        loss_=loss_tn/_cc
        if acc>=98:
            model.learning_rate=1e-5
        elif acc>=97:
            model.learning_rate=1e-4
#         elif acc>=95:
#             model.learning_rate=1e-4
        sam_tm=time()-sam_tm
        rem_sam=(60000-st)/btsz
        eta=int(rem_sam*sam_tm)
        print("\rProgress: {:.2f} %    Acc: {:.3f} %    loss: {:.6f}     Sample time: {:.3f}s    ETA: {}:{}s    .".format(perc,acc,loss_,sam_tm,eta//60,eta%60),end='')
        st+=btsz
    print("")

In [17]:
epochs=5
for epoch in range(epochs):
    print("EPOCH:",epoch+1,'/',epochs)
    st_tm=time()
    run()
    print("Epoch time: {}:{}s".format(int(time()-st_tm)//60,int(time()-st_tm)%60))
    validate()

EPOCH: 1 / 5
Progress: 100.00 %    Acc: 78.634 %    loss: 0.646396     Sample time: 0.001s    ETA: 0:0s    .
Epoch time: 0:8s
Validation Acc: 94.960 %        Val loss: 0.20122938
EPOCH: 2 / 5
Progress: 100.00 %    Acc: 95.271 %    loss: 0.185687     Sample time: 0.001s    ETA: 0:0s    .
Epoch time: 0:8s
Validation Acc: 95.000 %        Val loss: 0.19889822
EPOCH: 3 / 5
Progress: 100.00 %    Acc: 95.210 %    loss: 0.190971     Sample time: 0.001s    ETA: 0:0s    .
Epoch time: 0:9s
Validation Acc: 95.000 %        Val loss: 0.19754071
EPOCH: 4 / 5
Progress: 100.00 %    Acc: 95.188 %    loss: 0.186559     Sample time: 0.001s    ETA: 0:0s    .
Epoch time: 0:7s
Validation Acc: 95.040 %        Val loss: 0.19638438
EPOCH: 5 / 5
Progress: 100.00 %    Acc: 95.375 %    loss: 0.184525     Sample time: 0.017s    ETA: 0:0s    .
Epoch time: 0:8s
Validation Acc: 95.040 %        Val loss: 0.19532679
